In [1]:
import pandas as pd
from collections import defaultdict, deque
import heapq
from queue import PriorityQueue
df = pd.read_csv('tubedata.csv', header=None)
df.head()

station_dict = defaultdict(list)
zone_dict = defaultdict(set)

for index, row in df.iterrows():
  
  start_station = row[0]
  end_station = row[1]
  act_cost = int(row[3])

  zone1 = row[4]
  zone2 = row[5]

  # station dictionary of child station tuples (child_name, cost from parent to the child)
  # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
  station_list = station_dict[start_station]
  station_list.append((end_station, act_cost))

  # the following two lines add the other direction of the tube "step"
  station_list = station_dict[end_station]
  station_list.append((start_station, act_cost))
  
  # we add the main zone
  zone_dict[start_station].add(zone1)
  # we add the secondary zone
  if zone2 != "0":
    zone_dict[start_station].add(zone2)
    # if the secondary zone is not 0 it's the main zone for the ending station
    zone_dict[end_station].add(zone2)
  else:
    # otherwise the main zone for the ending station is the same as for the starting station
    zone_dict[end_station].add(zone1)

2.1 Implement DFS, BFS and UCS


In [2]:
def dfs(graph, start, goal, inverse_order=False):
    visited = set()
    stack = [(start, [start], 0)]  # station, path, cost
    expanded_nodes = 0
# Main loop  DFS
    while stack:
        (vert, path, cost) = stack.pop()
        expanded_nodes += 1

        if vert == goal:
            return path, cost, expanded_nodes
        visited.add(vert)

        neighbors = graph[vert]
        if inverse_order:
            neighbors = reversed(neighbors)

        for next, next_cost in neighbors:
            if next not in visited:
                stack.append((next, path + [next], cost + next_cost))

    return None


def bfs(graph, start, goal, inverse_order=False):
    queue = deque([(start, [start])])  # Queue of (current node, path)
    visited_nodes = set()
 # Main loop  BFS
    while queue:
        current, path = queue.popleft()

        if current == goal:
            return path  # Goal reached, return the path

        if current not in visited_nodes:
            visited_nodes.add(current)
            neigh = graph.get(current, [])

            if inverse_order:
                neigh = reversed(neigh)

            for neighbor in neigh:
                neighbor_name, _ = neighbor
                if neighbor_name not in visited_nodes:
                    queue.append((neighbor_name, path + [neighbor_name]))

    return None 
# Main loop ucs
def ucs(graph, start_station, goal_station, zone_dict, inverse_order=False):
    start_state = (0, start_station, [start_station])  # cost, station, path
    p_queue = [start_state]
    visited = set()

    while p_queue:
        current_cost, current_station, current_path = heapq.heappop(p_queue)

        if current_station == goal_station:
            return current_path, current_cost, len(visited) + 1  # +1 for the current node

        if current_station not in visited:
            visited.add(current_station)

            neighbors = graph[current_station]
            if inverse_order:
                neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

            for neighbor, cost in neighbors:
                if neighbor not in current_path:
                    new_cost = current_cost + cost
                    new_state = (new_cost, neighbor, current_path + [neighbor])
                    heapq.heappush(p_queue, new_state)

    return None
    
 
def routes(graph, start_station, end_station,inverse = False):
    print("Testing route from", start_station, "to", end_station)
    dfs_path, dfs_cost, dfs_expanded = dfs(graph, start_station, end_station,inverse)
    print("DFS:", dfs_path, dfs_cost, dfs_expanded)
    bfs_path= bfs(graph, start_station, end_station,inverse)
    bfs_cost = sum(graph[node1][0][1] for node1 in bfs_path[1:])
    bfs_exp_path = len(bfs_path)
    print("BFS:", bfs_path, bfs_cost, bfs_exp_path )
    ucs_path, ucs_cost, ucs_expanded = ucs(graph, start_station, end_station,zone_dict,inverse)
    print("UCS:", ucs_path, ucs_cost, ucs_expanded)

In [3]:
routes(station_dict, 'Canada Water',"Stratford")

Testing route from Canada Water to Stratford
DFS: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford'] 15 6
BFS: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford'] 15 6
UCS: ['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford'] 14 55


In [4]:
routes(station_dict, 'New Cross Gate',"Stepney Green")

Testing route from New Cross Gate to Stepney Green
DFS: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford', 'Mile End', 'Stepney Green'] 27 35
BFS: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green'] 12 8
UCS: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green'] 14 19


In [5]:
routes(station_dict, 'Ealing Broadway', "South Kensington")

Testing route from Ealing Broadway to South Kensington
DFS: ['Ealing Broadway', 'Ealing Common', 'North Ealing', 'Park Royal', 'Alperton', 'Sudbury Town', 'Sudbury Hill', 'South Harrow', 'Rayners Lane', 'West Harrow', 'Harrow-on-the-Hill', 'Northwick Park', 'Preston Road', 'Wembley Park', 'Finchley Road', 'Baker Street', 'Great Portland Street', 'Euston Square', "King's Cross St. Pancras", 'Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria', 'Sloane Square', 'South Kensington'] 67 194
BFS: ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington'] 16 9
UCS: ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington'] 19 50


In [6]:
routes(station_dict, 'Baker Street',"Wembley Park")

Testing route from Baker Street to Wembley Park
DFS: ['Baker Street', 'Finchley Road', 'Wembley Park'] 13 3
BFS: ['Baker Street', 'Finchley Road', 'Wembley Park'] 5 3
UCS: ['Baker Street', 'Finchley Road', 'Wembley Park'] 13 85


In [7]:
routes(station_dict, 'South Kensington', "Ealing Broadway")

Testing route from South Kensington to Ealing Broadway
DFS: ['South Kensington', 'Gloucester Road', "Earls' Court", 'Barons Court', 'Hammersmith', 'Turnham Green', 'Acton Town', 'Ealing Common', 'North Ealing', 'Park Royal', 'Alperton', 'Sudbury Town', 'Sudbury Hill', 'South Harrow', 'Rayners Lane', 'West Harrow', 'Harrow-on-the-Hill', 'Northwick Park', 'Preston Road', 'Wembley Park', 'Finchley Road', 'Baker Street', 'Great Portland Street', 'Euston Square', "King's Cross St. Pancras", 'Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Bond Street', 'Marble Arch', 'Lancaster Gate', 'Queensway', 'Notting Hill Gate', 'Bayswater', 'Paddington', 'Royal Oak', 'Westbourne Park', 'Ladbroke Grove', 'Latimer Road', "Shepherd's Bush", 'White City', 'East Acton', 'North Acton', 'West Acton', 'Ealing Broadway'] 108 227
BFS: ['South Kensington', 'Gloucester Road', "Earls' Court", 'Barons Court', 'Hammersmith', 'Turnham Green', 'Acton Town', 'Ealing Common', 'Ealing Broadway'] 17 9
UCS: ['So

In [8]:
## Inverse order

routes(station_dict, 'South Kensington', "Ealing Broadway",True)


Testing route from South Kensington to Ealing Broadway
DFS: ['South Kensington', 'Sloane Square', 'Victoria', "St. James' Park", 'Westminster', 'Embankment', 'Charing Cross', 'Piccadilly Circus', 'Oxford Circus', "Regent's Park", 'Baker Street', 'Marylebone', 'Edgware Road', 'Paddington', 'Bayswater', 'Notting Hill Gate', 'Holland Park', "Shepherd's Bush", 'White City', 'East Acton', 'North Acton', 'West Acton', 'Ealing Broadway'] 46 43
BFS: ['South Kensington', 'Gloucester Road', "Earls' Court", 'Barons Court', 'Hammersmith', 'Turnham Green', 'Acton Town', 'Ealing Common', 'Ealing Broadway'] 17 9
UCS: ['South Kensington', 'Gloucester Road', "Earls' Court", 'Barons Court', 'Hammersmith', 'Turnham Green', 'Acton Town', 'Ealing Common', 'Ealing Broadway'] 19 120


2.3 Extending the cost function

In [12]:
#2.3
station_lines = defaultdict(list)
for index, row in df.iterrows():
    start_station, end_station, line, act_cost, _, _ = row
    act_cost = int(act_cost)
    station_lines[start_station].append((end_station, (act_cost, line)))
    station_lines[end_station].append((start_station, (act_cost, line)))
def multi_line_change (graph, start, goal, line_change_cost=2):
    queue = [(0, start, [start], None)]  # Cost, station, path, current line
    expanded_nodes = 0
    visited_nodes = set()
    while queue:
        cost, vertex, path, current_line = heapq.heappop(queue)
        expanded_nodes += 1
        if vertex == goal:
            return path, cost, expanded_nodes
        if vertex not in visited_nodes:
            visited_nodes.add(vertex)
        for next_station, next_info in graph[vertex]:
            next_cost, next_line = next_info # Assuming the graph stores cost and line
            total_cost = cost + next_cost
            # Check for line change and add penalty if needed
            if current_line and next_line != current_line:
                total_cost+= line_change_cost
            if next_station not in path:
                heapq.heappush(queue, (total_cost, next_station, path + [next_station], next_line))
    return None
def uni_line_change(graph, s, e):
    # UCS with time change implementation
    ucs_path, ucs_start, ucs_end = multi_line_change(station_lines, s, e)
    print("UCS with line changes:", ucs_path, ucs_start, ucs_end)

In [13]:
routes(station_dict, 'South Kensington', "Stratford")

Testing route from South Kensington to Stratford
DFS: ['South Kensington', 'Gloucester Road', "Earls' Court", 'Barons Court', 'Hammersmith', 'Turnham Green', 'Acton Town', 'Ealing Common', 'North Ealing', 'Park Royal', 'Alperton', 'Sudbury Town', 'Sudbury Hill', 'South Harrow', 'Rayners Lane', 'West Harrow', 'Harrow-on-the-Hill', 'Northwick Park', 'Preston Road', 'Wembley Park', 'Finchley Road', 'Baker Street', 'Great Portland Street', 'Euston Square', "King's Cross St. Pancras", 'Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria', 'Pimlico', 'Vauxhall', 'Stockwell', 'Oval', 'Kennington', 'Elephant & Castle', 'Borough', 'London Bridge', 'Bank/Monument', 'Moorgate', 'Liverpool Street', 'Aldgate', 'Tower Hill', 'Aldgate East', 'Whitechapel', 'Stepney Green', 'Mile End', 'Bow Road', 'Bromley-by-Bow', 'West Ham', 'Stratford'] 122 122
BFS: ['South Kensington', 'Sloane Square', 'Victoria', "St. James' Park", 'Westminster', 'Waterloo', 'Bank/Monument', 'Liverpool Street', 'Be

In [15]:
uni_line_change(station_dict, 'Canada Water', "Stratford")

UCS with line changes: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford'] 15 84


Question 2.4 - Heuristic search

In [ ]:
#q2.4
#Heuristic
def heuristic(station, goal_station, zone_dict):
    if station in zone_dict and goal_station in zone_dict:
        station_zones = zone_dict[station]
        goal_zones = zone_dict[goal_station]
        station_zones = {int(zone) for zone in station_zones}
        goal_zones = {int(zone) for zone in goal_zones}
        min_zone_difference = min(abs(station_zone - goal_zone) for station_zone in station_zones for goal_zone in goal_zones)
        return min_zone_difference
    else:
        return 0  
def bfs(graph, start, goal, zone_dict):
    priority_queue = PriorityQueue()
    visited = set()
    priority_queue.put((heuristic(start, goal, zone_dict), start, [start]))
    while not priority_queue.empty():
        _, vertex, path = priority_queue.get()
        if vertex == goal:
            return path
        if vertex in graph and vertex not in visited:
            visited.add(vertex)
            for neighbor, _ in graph[vertex]:
                if neighbor not in path:
                    priority_queue.put((heuristic(neighbor, goal, zone_dict), neighbor, path + [neighbor]))
    return None
start_station = "Euston"
goal_station = "Victoria"
bfs_path = bfs(station_dict, start_station, goal_station, zone_dict)
print("Best First Search")
print("Path:", bfs_path)